In [1]:
import pandas as pd
import os 
import torch

In [2]:
import torchvision.transforms as transforms

In [3]:
import torchvision.datasets as datasets

In [4]:
global device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Device: {}'.format(device))

Device: cpu


In [5]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 64

In [6]:
# define training and test data directories

train_dir = 'C:/Users/ihelshal/Documents/Python Scripts/CatDogClassifier/Data/training_set/training_set'
test_dir = 'C:/Users/ihelshal/Documents/Python Scripts/CatDogClassifier/Data/test_set/test_set'

#create transformers
image_size = (224, 224)
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]
train_transform = transforms.Compose([
                                transforms.Resize(image_size), 
                                transforms.ToTensor(), 
                                transforms.Normalize(mean, std)])
test_transforms = transforms.Compose([
                                transforms.Resize(image_size), 
                                transforms.ToTensor(), 
                                transforms.Normalize(mean, std)])


 ## read data set using the custom class
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
dev_dataset = datasets.ImageFolder(root=test_dir, transform=test_transforms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(dev_dataset, 1, shuffle=False)


In [7]:
print('training Dataset =',len(train_dataset))
print('Testing Dataset =',len(dev_dataset))

training Dataset = 8005
Testing Dataset = 2023


In [8]:
import sys
#for batch_idx, (data, target) in enumerate(train_dataloader):
#    print(target)
#    if(batch_idx == 0):
#        sys.exit("Age less than 18")    
    

In [9]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy
from torch.utils.data import Dataset, DataLoader
import glob
from PIL import Image

In [10]:

class CNN_1(nn.Module):
    def __init__(self, input_size, n_feature, output_size):
        super(CNN_1, self).__init__()
        self.n_feature = n_feature
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=n_feature, kernel_size=5)
        self.conv2 = nn.Conv2d(n_feature, n_feature, kernel_size=5)
        self.fc1 = nn.Linear(n_feature*53*53, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x, verbose=False):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = x.view(-1, self.n_feature*53*53)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

class CNN_2(nn.Module):
    def __init__(self, input_size, n_feature, output_size):
        super(CNN_2, self).__init__()
        self.n_feature = n_feature
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=5)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=5)
        self.fc1 = nn.Linear(128*10*10, 50)
        self.fc2 = nn.Linear(50, 2)
       
    def forward(self, x, verbose=False):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv4(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)       
        x = x.view(-1, 128*10*10)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [12]:

accuracy_list = []
losses = []

def train(epoch, model):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        
        #print(data[0].shape)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if (batch_idx % 10 and  batch_idx >5):
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        
        losses.append(loss.item())
            
def test(model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss                                                               
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    accuracy_list.append(accuracy)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))


In [13]:
#function to count number of parameters
def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.nelement()
    return np

input_size  = 224*224*3   # images are 224*224 pixels and has 3 channels because of RGB color
output_size = 2      # there are 2 classes---Cat and dog

# Training settings  for model 1
# n_features = 2 # hyperparameter

# model_cnn1 = CNN_1(input_size, n_features, output_size)
# optimizer = optim.SGD(model_cnn1.parameters(), lr=0.01, momentum=0.5)
# print('Number of parameters: {}'.format(get_n_params(model_cnn1)))

# for epoch in range(0, 10):
#     train(epoch, model_cnn1)
#     test(model_cnn1)


# Training settings for model 2
n_features = 6 # hyperparameter
model_cnn2 = CNN_2(input_size, n_features, output_size)
optimizer = optim.SGD(model_cnn2.parameters(), lr=0.01, momentum=0.5)
print('Number of parameters: {}'.format(get_n_params(model_cnn2)))

for epoch in range(0, 10):
    train(epoch, model_cnn2)
    test(model_cnn2)

Number of parameters: 1308504
Train Epoch: 0 [384/8005 (5%)]	Loss: 0.692376
Train Epoch: 0 [448/8005 (6%)]	Loss: 0.704541
Train Epoch: 0 [512/8005 (6%)]	Loss: 0.669448
Train Epoch: 0 [576/8005 (7%)]	Loss: 0.694764
Train Epoch: 0 [704/8005 (9%)]	Loss: 0.691060
Train Epoch: 0 [768/8005 (10%)]	Loss: 0.701686
Train Epoch: 0 [832/8005 (10%)]	Loss: 0.685186
Train Epoch: 0 [896/8005 (11%)]	Loss: 0.691094
Train Epoch: 0 [960/8005 (12%)]	Loss: 0.680458
Train Epoch: 0 [1024/8005 (13%)]	Loss: 0.713717
Train Epoch: 0 [1088/8005 (13%)]	Loss: 0.699587
Train Epoch: 0 [1152/8005 (14%)]	Loss: 0.698364
Train Epoch: 0 [1216/8005 (15%)]	Loss: 0.674845
Train Epoch: 0 [1344/8005 (17%)]	Loss: 0.696756
Train Epoch: 0 [1408/8005 (17%)]	Loss: 0.703076
Train Epoch: 0 [1472/8005 (18%)]	Loss: 0.704057
Train Epoch: 0 [1536/8005 (19%)]	Loss: 0.694605
Train Epoch: 0 [1600/8005 (20%)]	Loss: 0.687020


KeyboardInterrupt: 

In [ ]:
torch.save(model_cnn2.state_dict(), 'dogcatwights_model2.pth')

In [ ]:
#model.load_state_dict(torch.load('checkpoint/epoch60_93.pt', map_location='cpu'))
#model_cnn1.eval()
#output = model_cnn1(list(test_loader)[0][0])
#_, predicted = torch.max(output, 1)
#if predicted[0].numpy() == 1:
#    print("dog")
#else:
#    print("cat")

In [22]:
import torch
import warnings
from PIL import Image
from torchvision import transforms

from torchvision import datasets, transforms, models

model = models.densenet121(pretrained=True)
print(model)


for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.1),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))

model.load_state_dict(torch.load('Output/dogcatwights_model3.pth', map_location='cpu'))
model.eval()

def image_transform(imagepath):
    test_transforms = transforms.Compose([transforms.Resize(255),
                                          transforms.CenterCrop(224),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                               [0.229, 0.224, 0.225])])

    
    image = Image.open(imagepath)
    imagetensor = test_transforms(image)
    return imagetensor


def predict(imagepath):
    model.eval()

    image = image_transform(imagepath)
    image1 = image[None,:,:,:]
    ps=torch.exp(model(image1))
    topconf, topclass = ps.topk(1, dim=1)
    if topclass.item() == 1:
        return {'class':'dog','confidence':str(topconf.item())}
    else:
        return {'class':'cat','confidence':str(topconf.item())}

In [25]:
print(predict('Data/TestOne/cat2.jpg'))

{'class': 'cat', 'confidence': '0.8557226657867432'}
